#### Carregar Datasets

In [1]:
import pandas as pd

In [2]:
previous_application = pd.read_csv('/home/jean/Documents/Analise-de-Dados/Analise-de-Dados/df/previous_application.csv', nrows=100000)
application_data = pd.read_csv('/home/jean/Documents/Analise-de-Dados/Analise-de-Dados/df/application_data.csv')

In [3]:
previous_application.shape, previous_application.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 37 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   SK_ID_PREV                   100000 non-null  int64  
 1   SK_ID_CURR                   100000 non-null  int64  
 2   NAME_CONTRACT_TYPE           100000 non-null  object 
 3   AMT_ANNUITY                  78867 non-null   float64
 4   AMT_APPLICATION              100000 non-null  float64
 5   AMT_CREDIT                   100000 non-null  float64
 6   AMT_DOWN_PAYMENT             49416 non-null   float64
 7   AMT_GOODS_PRICE              78400 non-null   float64
 8   WEEKDAY_APPR_PROCESS_START   100000 non-null  object 
 9   HOUR_APPR_PROCESS_START      100000 non-null  int64  
 10  FLAG_LAST_APPL_PER_CONTRACT  100000 non-null  object 
 11  NFLAG_LAST_APPL_IN_DAY       100000 non-null  int64  
 12  RATE_DOWN_PAYMENT            49416 non-null   float64
 13  

((100000, 37), None)

In [4]:
application_data.shape, application_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


((307511, 122), None)

#### Merger Datasets

In [5]:
df = application_data.merge(previous_application, left_on='SK_ID_CURR', right_on='SK_ID_CURR', how='inner')

In [6]:
df.shape

(84752, 158)

In [7]:
#Lista de Objetos
objList = df.select_dtypes(include='object').columns
print(objList)

Index(['NAME_CONTRACT_TYPE_x', 'CODE_GENDER', 'FLAG_OWN_CAR',
       'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE_x', 'NAME_INCOME_TYPE',
       'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE',
       'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START_x', 'ORGANIZATION_TYPE',
       'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE',
       'EMERGENCYSTATE_MODE', 'NAME_CONTRACT_TYPE_y',
       'WEEKDAY_APPR_PROCESS_START_y', 'FLAG_LAST_APPL_PER_CONTRACT',
       'NAME_CASH_LOAN_PURPOSE', 'NAME_CONTRACT_STATUS', 'NAME_PAYMENT_TYPE',
       'CODE_REJECT_REASON', 'NAME_TYPE_SUITE_y', 'NAME_CLIENT_TYPE',
       'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE',
       'CHANNEL_TYPE', 'NAME_SELLER_INDUSTRY', 'NAME_YIELD_GROUP',
       'PRODUCT_COMBINATION'],
      dtype='object')


In [8]:
#Soma de dados faltantes (isnull ou isna)
df.isnull().sum()

SK_ID_CURR                       0
TARGET                           0
NAME_CONTRACT_TYPE_x             0
CODE_GENDER                      0
FLAG_OWN_CAR                     0
                             ...  
DAYS_FIRST_DUE               31799
DAYS_LAST_DUE_1ST_VERSION    31799
DAYS_LAST_DUE                31799
DAYS_TERMINATION             31799
NFLAG_INSURED_ON_APPROVAL    31799
Length: 158, dtype: int64

In [9]:
dados_faltantes_percentual = df.isna().sum()*100/len(df)
print(dados_faltantes_percentual)

#filtra colunas com mais de 20% de dados faltando
dados_faltando_perc_columns = dados_faltantes_percentual[dados_faltantes_percentual>=20]
dados_faltando_perc_columns.count()

SK_ID_CURR                    0.000000
TARGET                        0.000000
NAME_CONTRACT_TYPE_x          0.000000
CODE_GENDER                   0.000000
FLAG_OWN_CAR                  0.000000
                               ...    
DAYS_FIRST_DUE               37.520059
DAYS_LAST_DUE_1ST_VERSION    37.520059
DAYS_LAST_DUE                37.520059
DAYS_TERMINATION             37.520059
NFLAG_INSURED_ON_APPROVAL    37.520059
Length: 158, dtype: float64


64

In [10]:
#Filtrar dados faltantes

#Removendo com mais de 20% de dados faltando
filtrado_df = df.drop(columns=dados_faltando_perc_columns.index)


In [11]:
#Preenchendo com a função backfill ou ffill

filtrado_df = filtrado_df.fillna(method='backfill')

In [12]:
filtrado_df.isna().sum()

SK_ID_CURR              0
TARGET                  0
NAME_CONTRACT_TYPE_x    0
CODE_GENDER             0
FLAG_OWN_CAR            0
                       ..
CHANNEL_TYPE            0
SELLERPLACE_AREA        0
NAME_SELLER_INDUSTRY    0
NAME_YIELD_GROUP        0
PRODUCT_COMBINATION     0
Length: 94, dtype: int64

#### Modelo Preditivo

In [13]:
# Bibliotecas para o modelo e feature engineering
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import (roc_curve,
                             recall_score,
                             accuracy_score,
                             roc_auc_score,
                             precision_score,
                             f1_score)

from sklearn.model_selection import KFold 

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.ensemble import (AdaBoostClassifier, 
                              GradientBoostingClassifier, 
                              ExtraTreesClassifier,
                              RandomForestClassifier)
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb

##### Ajustando os dados

In [14]:
le = preprocessing.LabelEncoder()

In [15]:
# Transformar variaveis categoricas em numericas (Sim e não em 0 e 1)
# Ainda não entendi essa transformação

cat_cols = filtrado_df.select_dtypes(include='object').columns #Variavel com pouca informação

for i in cat_cols:
    filtrado_df[i] = le.fit_transform(filtrado_df[i])

In [16]:
# Colocando SK_ID_CURR como index

filtrado_df.set_index('SK_ID_CURR', inplace = True) 
